In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel
from google.colab import drive, files
import torch
import os
import pandas as pd

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_dir = "drive/My Drive/Colab Notebooks/"
#sanity check. Does it replicate prep_filler_gap material in Hu? Yes
file_location1 = "fgd_pp_input.xlsx"
file_location2 = "period/fgd_pp_input.xlsx"
#wh_island    separate files of testing materials so that colab doesn't crush
file_location3 = "wh-island_input.xlsx"
file_location4 = "wh-island_input2.xlsx"
#adjunct_island  separate files of testing materials so that colab doesn't crush
file_location5 = "adjunct-islands_input.xlsx"
file_location6 = "adjunct-islands_input2.xlsx"
file_location7 = "adjunct-islands_input3.xlsx"
file_location8 = "adjunct-islands_input4.xlsx"
#complexNP_island
file_location11 = "complexNP_input.xlsx"
file_location12 = "complexNP_input2.xlsx"
file_location13 = "complexNP_input3.xlsx"
file_location14 = "complexNP_input4.xlsx"
file_location15 = "complexNP_input5.xlsx"
file_location16 = "complexNP_input6.xlsx"
file_location17 = "complexNP_input7.xlsx"
file_location18 = "complexNP_input8.xlsx"

In [ ]:
df = pd.read_excel(root_dir+file_location3)

In [ ]:
def formatData(df):
    prefixes = df.iloc[:,:-1].agg(" ".join, axis=1)
    continuation = df["continuation"]
    combined_df = pd.DataFrame({"prefix": prefixes, "continuation": continuation})

    new_df = pd.DataFrame(columns=["prefix", "continuation"])

    for i, row in combined_df.iterrows():

        next_words = row["continuation"].split(" ")
        new_prefix = row["prefix"]

        new_df = new_df.append(pd.Series({"prefix": new_prefix, "continuation": next_words[0]}), ignore_index=True)

        for i in range(len(next_words)-1):
            new_prefix = new_prefix + " " + next_words[i]
            new_df = new_df.append(pd.Series({"prefix": new_prefix, "continuation": next_words[i+1]}), ignore_index=True)

    return new_df


In [ ]:
formatted_df = formatData(df)

<ipython-input-75-255d6454f6c8>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series({"prefix": new_prefix, "continuation": next_words[0]}), ignore_index=True)
<ipython-input-75-255d6454f6c8>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series({"prefix": new_prefix, "continuation": next_words[i+1]}), ignore_index=True)
<ipython-input-75-255d6454f6c8>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(pd.Series({"prefix": new_prefix, "continuation": next_words[i+1]}), ignore_index=True)
<ipython-input-75-255d6454f6c8>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = "left"
#model = GPT2LMHeadModel.from_pretrained('drive/My Drive/Colab Notebooks/GPT2_checkpoints/mix_construction/model_0_lr_.00000625_26')
#model = GPT2LMHeadModel.from_pretrained('drive/My Drive/Colab Notebooks/GPT2_checkpoints/adjunct_island/model_0_lr_.00000625_102')

#model = GPT2LMHeadModel.from_pretrained('drive/My Drive/Colab Notebooks/GPT2_checkpoints/wh_island/model_1_lr_.00000625_250')
#model = GPT2LMHeadModel.from_pretrained('drive/My Drive/Colab Notebooks/GPT2_checkpoints/adjunct_island/model_1_lr_.00000625_250')
#model = GPT2LMHeadModel.from_pretrained('drive/My Drive/Colab Notebooks/GPT2_checkpoints/complexNP_island/model_1_lr_.00000625_250')
model = GPT2LMHeadModel.from_pretrained('drive/My Drive/Colab Notebooks/GPT2_checkpoints/mix_construction/model_0_lr_.00000625_65')
#model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
print(formatted_df.iloc[1])

prefix          The teacher realized that the student in the f...
continuation                                                great
Name: 1, dtype: object


In [ ]:
model.eval()
df = formatted_df
df = df.copy()
df["continuation"] = " " + df["continuation"]
prefixes = df["prefix"].tolist()
conts = df["continuation"].tolist()
surprisals = []
for i, j in zip(prefixes, conts):
  encoded_inputs = tokenizer.encode(i, padding=True)
  tokens_tensor = torch.tensor([encoded_inputs])
  conts_index = torch.tensor([tokenizer.encode(j,padding=True)]).item()
  with torch.no_grad():
    outputs = model(tokens_tensor)
    conditional_probability = torch.nn.functional.softmax(outputs.logits[:,-1,:], dim=1)[0][conts_index]
  surprisals.append(-torch.log(conditional_probability).item())

df["surprisals"]=surprisals   

In [ ]:
df.to_csv('.csv')
files.download('.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
display(df)

In [ ]:
model.eval()
encoded_inputs = tokenizer.encode("I want a cup of coffee", padding=True)
#cont = tokenizer.encode("cream milk . shoes", padding=True)
#print(cont)
tokens_tensor = torch.tensor([encoded_inputs])
outputs = model(tokens_tensor)[0]
#print(outputs)
k100 = torch.topk(outputs[0, -1, :],k=100, sorted = True)
predicted_indices = k100.indices
predicted_text = tokenizer.decode(predicted_indices)
print(predicted_text)

 that't with. who had for out wasn teacher can found weren teachers couldn wouldn didn could what some will might would should doesn was knows has doctors allresses find discovered this hadn figures isn finds areners knew these like realizedals are guy didressians podologists most kissed cups about is boy schools every boys ladies girlistsian realizes lady dropped Churchillleyen break revealed forgot know haven manymen students won looks lot realizeiat talked company doctor made visits wereops goes those driverologist glasses discovers sweep haveors
